In [1]:
%pylab
from scipy.integrate import odeint
from scipy.optimize import brentq
import math

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
# One conduit constriction model
# Constants - unchanging
rho_w = 1000.#density of water, kg/m^3
rho_i = 900#density of ice, kg/m^3
L_f = 3.34e5#Latent heat of fusion (ice), J/kg
n=3.#ice flow law exponent, unitless
B=5.8e7#Arrhenius parameter, N/m^2 *s^(1/n)
f=0.1#Darcy-Weisbach friction factor, unitless
g = 9.8#gravitational accel, m^2/s
P = 24.*3600. # Period - 1 day in seconds

In [3]:
# Functions
def R(ti,Ri):
    P=24.*3600.
    return Ri*(1.+sin(2.*pi*ti/P)/2.)

def P_wet(A):
    return 2.*sqrt(pi*A)#2.*A**2./pi
def D_H(A):
    return 4.*A/P_wet(A)
def C(A,L):
    return A*sqrt(2.*g/(1+f*L/D_H(A)))
# For a single conduit
def Qh(hi,A):
    return A*sqrt(2.*g*abs(hi)/(1+f*Li/D_H(A)))*sign(hi)
def dy_dt1(y,t,Ri,Zi):
    Pi = rho_i*g*Zi
    hi = y[0]
    Ai = y[1]
    hout = 0
    Q1 = Qh(hi-hout,Ai)
    dh_dt = (Ri - Q1)/A_R
    Pw = rho_w*g*hi
    dA_dt = f*rho_w/(8*rho_i*L_f)*P_wet(Ai)*(Q1/Ai)**3 - 2.*(1./(n*B))**n*Ai*(Pi-Pw)*abs(Pi-Pw)**(n-1.)
    return [dh_dt,dA_dt]

def num_subplots(n):
    if n <= 4:
        i=n;j=1
    elif n <=8:
        i=2;j=4
    elif n <=12:
        i=3;j=4
    elif n <=16:
        i=4;j=4
    elif n <=20:
        i=5;j=4
    elif n <=24:
        i=6;j=4
    else:
        print 'Number of subplots too large'
    return(i,j)

def det_idx(k):
    if k <= 3:
        i=0;j=k
    elif k <= 7:
        i=1;j=k-4
    elif k <= 11:
        i=2;j=k-8
    elif k <= 15:
        i=3;j=k-12
    elif k <= 19:
        i=4;j=k-16
    elif k <= 23:
        i=5;j=k-20
    else:
        print 'Number of Plots is too large'
    return(i,j)

In [18]:
# Investigating: minimum hydraulic diameter neccessary
# Changing:


# plots for one conduit model
Li = 5000.
# Ai = 
Ai = linspace(0.01,1,num=10) # Initial cross sectional area
# A_min=0.1
hi = 300.
Ri = 0.8


A_R= 50# Moulin cross sectional area
Z=500 # Ice thickness

# Run model with above specification
nhours = 500
nsecs = nhours*60*60
t = linspace(0,nsecs,500)
tplot=t/3600.


# find the minimum A0 that is physically possible
sol1_list = []
A_list = []
R_list = []


for A in Ai:
    sol_test = odeint(dy_dt1,[hi,A],t,args=(R,Z))
    sol1_list.append(sol_test)
    if max(sol_test[:,0]) < Z:
        A_list.append(A)
#             print R, A
# print A_list
# print len(A_list)
# A_min = min(Amin_list)

#change it to plot everything instead of the min


i,j=num_subplots(len(A_list))
print i,j
fig, axs=subplots(nrows=i,ncols=j)
fig.set_tight_layout(True)


sol=[]
k=0
if len(A_list) <= 4:
    for A in A_list:
        sol=odeint(dy_dt1, [hi,A], t, args=(Ri,Z))
        ax=axs[k]
        ax2 = ax.twinx()
        ax.plot(tplot,sol[:,0])
        Rec1 = Ri*(1.+sin(2.*pi*t/P)/2.)
        ax2.plot(tplot,Rec1,'k--')
#         ax.set_xlabel("Time (hours)")
        ax.set_title("A="+str(A))
#         ax.set_ylabel("Moulin Head")
#         ax2.set_ylabel("Recharge")
        k=k+1
else:
    for A in A_list:
        sol=odeint(dy_dt1, [hi,A], t, args=(Ri,Z))
        idx1,idx2 = det_idx(k)
        
        ax=axs[idx1,idx2]
        ax2 = ax.twinx()
        ax.plot(tplot,sol[:,0])
        Rec1 = Ri*(1.+sin(2.*pi*t/P)/2.)
        k=k+1
        ax2.plot(tplot,Rec1,'k--')
#         ax.set_xlabel("Time (hours)")
        ax.set_title("A="+str(A))
#         ax.set_ylabel("Moulin Head")
#         ax2.set_ylabel("Recharge")




# # print model run specifications
# print 'Single Conduit Model'
# print 'Max Head =',max(sol1[:,0]), 'Min Head =',min(sol1[:,0])
# print 'Minimum starting A_c =',A_min,'Equilibrium A_c =',sol1[-1,1]
# print 'Average Recharge =',Ri,'Moulin Area =',A_R,'Ice thickness =',Z
# # Plotting above version of the model
# tplot=t/3600.
# fig, ax1 = subplots()
# ax2 = ax1.twinx()
# ax1.plot(tplot,sol1[:,0])
# ax2.plot(tplot,Rec1,'k--')
# ax1.set_xlabel("Time (hours)")
# ax1.set_ylabel("Moulin Head ($m$)")
# ax2.set_ylabel("Recharge ($m^3/s$)")
# # A_c
# figure()
# plot(tplot,sol1[:,1])
# xlabel("Time (hours)")
# ylabel("Cross-sectional Area ($m^2$)")


# %xdel sol1

2 4


In [7]:
len(A_list)

8

In [8]:
A_list

[0.23000000000000001,
 0.34000000000000002,
 0.45000000000000001,
 0.56000000000000005,
 0.67000000000000004,
 0.78000000000000003,
 0.89000000000000001,
 1.0]

In [ ]:
fig, ax1 = subplots()
ax2 = ax1.twinx()
ax1.plot(tplot,sol1[:,0])
ax2.plot(tplot,Rec1,'k--')
ax1.set_xlabel("Time (hours)")
ax1.set_ylabel("Moulin Head ($m$)")
ax2.set_ylabel("Recharge ($m^3/s$)")

In [ ]:
x=1; y=2

In [ ]:
y

In [ ]:
len(Amin_list)

In [ ]:
j